<img src="files/images/Synaspe_newLogo.png" width="200" height="260" align="left"/>
<br>
<br>

### MTB Webinar Jan 15, 2019
_Abhi Pratap & Larsson Omberg_

#### Goals 
* Explore a simple survey data and associated metadata collected and stored in synapse
* Score the survey and store results back in synapse

#### 1. Connect and log-in to the system
[More details](https://python-docs.synapse.org/build/html/index.html)

In [2]:
import synapseclient
syn = synapseclient.login()

Welcome, Abhishek Pratap!



For demo purposes we will explore the data from a recent mobile study that used three short form versions of Neuro-QOLs. Specifically we will look at the 8-item Neuro-QOL survey for the Lower Extremity.
[Explore data on synapse](https://www.synapse.org/#!Synapse:syn10142944/tables/)

#### 2. Fetch raw data and meta data from Synapse  

In [6]:
neuroQOL_lowExtremity = syn.tableQuery("select * from syn10142944")
neuroQOL_lowExtremity

In [7]:
syn.tableQuery?